# Obtención Predicciones por Producto y País

1. [Carga del DataFrame](#carga-del-dataframe)
<br></br>
2. [Función que Extrae Datos por País y Producto](#funcion-que-extrae-datos-por-pais-y-producto)
<br></br>
3. [Generación de la Predicción](#generacion-de-la-prediccion)
<br></br>
4. [Conclusiones](#conclusiones)

In [1]:
%load_ext watermark
%watermark

2019-05-29T13:30:58+02:00

CPython 3.6.8
IPython 6.2.1

compiler   : GCC 7.3.0
system     : Linux
release    : 4.13.0-46-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 2
interpreter: 64bit


## 1. <a id='carga-del-dataframe'>Carga del DataFrame</a>

In [1]:
import math
import numpy as np
import pandas as pd
from datetime import datetime,timedelta
import calendar
import matplotlib.pyplot as plt
from fbprophet import Prophet
import warnings

%matplotlib inline
plt.rcParams['figure.figsize'] = (18,10)
warnings.filterwarnings('ignore')

ERROR:fbprophet:Importing plotly failed. Interactive plots will not work.


In [3]:
df = pd.read_pickle('../Data/01.-Exploracion-de-los-Datos-y-Creacion-DataFrame.pkl')

df.head()

,Fecha_envio,Product_code,Origen,Destino,Peso,Volumen,Peso_final
0,2017-05-01,B,ES,NO,1.80,3.584,3.584
1,2017-05-01,B,ES,NO,0.90,2.242,2.242
2,2017-05-01,B,ES,NO,0.35,1.009,1.009
3,2017-05-01,B,ES,NO,1.15,2.130,2.130
4,2017-05-01,B,ES,NO,0.45,0.996,0.996


## 2. <a id='funcion-que-extrae-informacion-por-pais-y-producto'>Función que Extrae Datos por País y Producto</a>

In [4]:
def dataframe_creation(product,country):
    df_temp = df[(df['Product_code'] == product) & (df['Destino'] == country)]
    df_temp = df_temp.groupby(['Fecha_envio']).size().reset_index()
    return df_temp

In [5]:
df_N_ES = dataframe_creation('N','ES')

df_N_ES.head()

,Fecha_envio,0
0,2017-05-02,1993
1,2017-05-03,3193
2,2017-05-04,2759
3,2017-05-05,2503
4,2017-05-06,56


## 3. <a id='generacion-de-la-prediccion'>Generación de la Predicción</a>

Utilizamos la función creada en el Notebook "03.-Implementacion-Prophet-a-df_U_FR.ipynb":

In [6]:
def new_y_for_date(date,df_temp):
    df_monthdays_for_date = df_temp.loc[(df_temp['ds'].dt.year == date.year) &
                                        (df_temp['ds'].dt.month == date.month) &
                                        (df_temp['ds'].dt.weekday == date.weekday())]
    new_y = df_monthdays_for_date['y'].mean()
    if math.isnan(new_y) == False:
        return np.int64(round(new_y))
    else:
        return np.int64(round(df_temp.loc[df_temp['ds'].dt.weekday == date.weekday()]['y'].mean()))

Generamos una función a partir de snippets de código del mismo Notebook, que nos genera un dataframe completo y preparado para ser procesado por Prophet:

In [7]:
def dataframe_creation(product,country):
    df_temp = df[(df['Product_code'] == product) & (df['Destino'] == country)]
    df_temp = df_temp.groupby(['Fecha_envio']).size().reset_index()
    df_temp.columns = ['ds','y']
    missing_dates = pd.date_range(start = '2017-05-01',end = '2019-04-01').difference(df_temp['ds'])
    df_missing_dates = pd.DataFrame()
    df_missing_dates['ds'] = missing_dates
    df_missing_dates['y'] = [new_y_for_date(date,df_temp) for date in df_missing_dates['ds']]
    df_temp = df_temp.append(df_missing_dates)
    df_temp['ds'] = pd.to_datetime(df_temp['ds'])
    df_temp = df_temp.sort_values(by = 'ds')
    df_temp = df_temp.reset_index(drop = True)
    return df_temp

Creamos una última función que genera la predicción a partir únicamente del 'Product_code', 'Destino' y el country_name_code, el ajusta a Prophet para que tenga en cuenta los días de vacaciones del país con el queramos trabajar:

In [8]:
def forecast(product,country,country_name_code):
    f = Prophet()
    f.add_country_holidays(country_name = country_name_code)
    f.fit(dataframe_creation(product,country))
    df_temp = f.make_future_dataframe(periods = 60)
    df_temp = f.predict(df_temp)
    return df_temp

Comprobamos que funciona correctamente:

In [9]:
df_U_FR = forecast('U','FR','FRA')

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [10]:
df_U_FR.head()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,Armistice 1918,Armistice 1918_lower,Armistice 1918_upper,Armistice 1945,...,holidays,holidays_lower,holidays_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2017-05-01,1676.917583,-423.900495,984.638915,1676.917583,1676.917583,0.0,0.0,0.0,0.0,...,-2559.694664,-2559.694664,-2559.694664,1117.779763,1117.779763,1117.779763,0.0,0.0,0.0,235.002682
1,2017-05-02,1677.012686,1643.955099,3104.443124,1677.012686,1677.012686,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,698.381247,698.381247,698.381247,0.0,0.0,0.0,2375.393933
2,2017-05-03,1677.107790,1607.348722,2993.301981,1677.107790,1677.107790,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,665.281319,665.281319,665.281319,0.0,0.0,0.0,2342.389109
3,2017-05-04,1677.202894,1602.871395,3020.237233,1677.202894,1677.202894,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,632.842299,632.842299,632.842299,0.0,0.0,0.0,2310.045193
4,2017-05-05,1677.297997,1208.394825,2576.724392,1677.297997,1677.297997,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,192.839190,192.839190,192.839190,0.0,0.0,0.0,1870.137188


## 3. <a id='conclusiones'>Conclusiones</a>